In [4]:
# made by @elasmobranches
import os
import glob
from PIL import Image, ImageDraw

# Class ID와 RGB 색상 mapping
color_map = {
    0: (255, 0, 0),     # Red: Flower
    1: (0, 255, 0),     # Green: Immature_fruit
    2: (0, 0, 255),     # Blue: Leaf
    3: (255, 255, 0),   # Yellow: Mature_fruit
    4: (255, 0, 255),   # Pink: Stem
}

def yolo_seg_to_mask(txt_path, img_path, output_path):
    """
    YOLO Segmentation Annotation을 RGB 마스크로 변환

    Args:
        txt_path: YOLO Annotation txt 파일 경로
        img_path: 원본 이미지 파일 경로
        output_path: 출력할 mask 이미지 경로
    """
    # 원본 이미지 크기 가져오기
    img = Image.open(img_path)
    width, height = img.size

    # 마스크 이미지 생성 (검정 배경, RGB 모드)
    mask = Image.new('RGB', (width, height), (0,0,0))   # default: 검정
    draw = ImageDraw.Draw(mask)

    # YOLO annotation 파일 열기
    with open(txt_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        values = list(map(float, line.strip().split()))
        class_id = int(values[0])   # Class ID (0부터 시작)

        # Class ID가 mapping되지 않은 경우 무시
        if class_id not in color_map:
            print(f"Warning: Class ID {class_id} is not in color_map. Skipping.")
            continue

        # 좌표 쌍 추출
        coordinates = values[1:]    # 첫 번째 값은 Class ID이므로 제외

        # 좌표를 pixel 값으로 변환할 때 수정
        polygon_coords = []
        for i in range(0, len(coordinates), 2):
            # 정규화된 좌표가 1.0인 경우 보정
            x = min(int(coordinates[i] * width), width - 1)
            y = min(int(coordinates[i+1] * height), height - 1)
            polygon_coords.append((x, y))

            # 좌표 유효성 검사
            if 0 <= x < width and 0 <= y < height:
                polygon_coords.append((x, y))
            else:
                print(f"Warning: Skipping invalid coordinate ({x}, {y})")
        
        # Class ID에 해당하는 RGB 색상
        pixel_color = color_map[class_id]
        
        # Polygon 그리기
        if len(polygon_coords) > 2:     # polygon을 위해선 최소 3개의 점 필요
            draw.polygon(polygon_coords, fill=pixel_color)
    
    # 마스크 저장
    mask.save(output_path, 'PNG')
    print(f"Saved mask to {output_path}")

def convert_dataset(yolo_dir, images_dir, output_dir):
    """
    데이터셋 전체를 변환한다.

    Args:
        yolo_dir: YOLO annotation 파일이 있는 디렉터리
        images_dir: 원본 이미지가 있는 디렉터리
        output_dir: 변환된 마스크를 저장할 디렉터리
    """
    os.makedirs(output_dir, exist_ok=True)

    # 모든 txt 파일에 대해 변환 수행
    for txt_path in glob.glob(os.path.join(yolo_dir, '*.txt')):
        # 이미지 파일명 추출
        base_name = os.path.splitext(os.path.basename(txt_path))[0]

        # 대응하는 이미지 파일 경로
        img_path = os.path.join(images_dir, f'{base_name}.jpg')
        if not os.path.exists(img_path):
            img_path = os.path.join(images_dir, f'{base_name}.png')

        if not os.path.exists(img_path):
            print(f"Warning: No image found for {base_name}")
            continue

        # 출력 마스크 파일 경로
        mask_path = os.path.join(output_dir, f'{base_name}_mask.png')

        try:
            # 변환 수행
            yolo_seg_to_mask(txt_path, img_path, mask_path)
        except Exception as e:
            print(f"Error converting {base_name}: {str(e)}")


In [ ]:
for i in ['train', 'valid', 'test']:
    yolo_dir = f"C:/Users/dohyeon/Downloads/rf_dataset_yolov11/{i}/labels"
    images_dir = f"C:/Users/dohyeon/Downloads/rf_dataset_yolov11/{i}/images"
    output_dir = f"C:/Users/dohyeon/Downloads/rf_dataset_yolov11/{i}/masks"

    convert_dataset(yolo_dir, images_dir, output_dir)